# fit central density and temperature to the given mass of the star 

Date of creation: 13.07.2020

Last updated: 13.07.2020

In [1]:
import math
import numpy as np
import pandas as pd

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import NotebookLoader
import Physics as ph
import SSM18 as s
import Rmodel as r

importing Jupyter notebook from Physics.ipynb
Physics version 1.14 13.07.2020
importing Jupyter notebook from SSM18.ipynb
SSM18 version 1.9 13.07.2020
importing Jupyter notebook from Rmodel.ipynb
Rmodel version 1.06 16.07.2020


In [3]:
def load_hydrogen():
    filename = 'Mass/hydrogen.csv'
    return pd.read_csv(filename)
epoch = 0
dfx = load_hydrogen()
dimM = len(dfx.index)
dfx

,Unnamed: 0,Epoch00,Epoch01
0,0,0.732,0.648658
1,1,0.732,0.652529
2,2,0.732,0.654702
3,3,0.732,0.656470
4,4,0.732,0.658008
...,...,...,...
996,996,0.732,0.732000
997,997,0.732,0.732000
998,998,0.732,0.732000
999,999,0.732,0.732000


In [4]:
epoch = 0
hyd = dfx['Epoch%02d' % epoch].values

In [5]:
"""
m is M/s.M (must be between 0 and 1)
"""
def XYZ(m, Z=0.02):
    i = int(m*(len(hyd) - 1))
    if i > len(hyd) - 1:
        i = len(hyd) - 1
    X = hyd[i]
    Y = 1 - Z - X
    return (X, Y, Z)
    

In [6]:
XYZ(0)

(0.732, 0.248, 0.02)

In [7]:
%%time
t0=1.33300e+07 
d0=94.5544713300
n, M, R, L, T, df = r.model_R(d0, t0, XYZ, debug=False)
n, M/s.M, R/s.R, L/s.L, T

Wall time: 543 ms


(9443,
 0.7105269551847768,
 0.3986475114357039,
 0.5218511789074138,
 7778.322419247444)

### Поиск комбинации центральных плотности и температуры, дающих решение с массой равной солнечной. (Первый вариант)

Для интегрирования уравнений из центра надо задать плотность и температуру в центре, но только одна их комбинации 
соответствует решению с массой, равной солнечной (при фиксированном хим. составе)

Внешний цикл по температурам, внутренний цикл - по плотности.
При фиксированной температуре постепенно повышаем плотность, пока не появится стабильное решение с массой < 1.
После этого возвращаемся на 1 шаг назад по плотности, уменьшаем шаг по плотности в 10 раз и повторям процедуру.
Так делаем пока шаг по плотности не станет меньше 1e-8.

In [8]:
%%time
dim = 11
for j in range(dim) :
#     T0 = 1.333e7 + 0.001e7*j  # M = M_sol, epoch = 0
    T0 = 14.51e6 + 0.01e6*j
    step = 10 # step by density
    exc = True
    den0 = 60
    if j > 0:
        df_prev = df.copy()
    while step > 1e-8 :    
        for i in range(10) :
#             print('j=%d i=%d step=%.9f d=%.8f t=%.0f' % (j, i, step, den0, T0))
            try :
                n, M, R, L, T, df = r.model_R(den0, T0, XYZ, debug=False)
            except Exception as error:
#                 print(T0, den0, 'Caught this error: ' + repr(error))
                exc = True
                den0 += step
                break
            if exc :
                exc = False
                print('\tT0=%.5e D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/s.M, R/s.R, L/s.L, T))
                den0 -= step
                step /= 10.
                break
            den0 -= step
    print('# T0=%.5e D0=%.2f n=%d M=%.4f R=%.4f L=%.3f T=%.0f' % (T0, den0, n, M/s.M, R/s.R, L/s.L, T))
    if M/s.M > 1 :
#         df = df_prev
        break
print("Done!")

	T0=1.45100e+07 D0=100.0000000000 n=9396 M=0.7847 R=0.4044 L=0.8860 T=8817
	T0=1.45100e+07 D0=91.0000000000 n=9458 M=0.8367 R=0.4293 L=0.8660 T=8510
	T0=1.45100e+07 D0=90.1000000000 n=9682 M=0.9579 R=0.5195 L=0.9245 T=7860
	T0=1.45100e+07 D0=90.0900000000 n=9777 M=0.9822 R=0.5667 L=0.9260 T=7532
	T0=1.45100e+07 D0=90.0890000000 n=9832 M=0.9899 R=0.5920 L=0.9262 T=7372
	T0=1.45100e+07 D0=90.0886000000 n=9879 M=0.9973 R=0.6323 L=0.9262 T=7127
	T0=1.45100e+07 D0=90.0885600000 n=9890 M=0.9994 R=0.6538 L=0.9262 T=7015
	T0=1.45100e+07 D0=90.0885520000 n=9914 M=1.0004 R=0.6729 L=0.9262 T=6914
	T0=1.45100e+07 D0=90.0885518000 n=9909 M=1.0005 R=0.6763 L=0.9262 T=6896
	T0=1.45100e+07 D0=90.0885517400 n=9916 M=1.0006 R=0.6809 L=0.9262 T=6872
# T0=1.45100e+07 D0=90.09 n=9916 M=1.0006 R=0.6809 L=0.926 T=6872
Done!
Wall time: 1min


In [9]:
# df = df_prev
df

,Mass,Radius,Temperature,Density,Luminosity,Hydrogen,Opacity,Convection,Pressure,Velocity
0,0.000000e+00,0.000000e+00,1.451000e+07,90.088552,0.000000e+00,0.732,1.445585,0.0,1.794413e+17,1.208950e-10
1,7.513729e+23,1.258054e+07,1.451000e+07,90.088462,1.298535e+25,0.732,1.445584,0.0,1.794411e+17,1.208948e-10
2,6.010983e+24,2.516108e+07,1.450999e+07,90.088353,1.038826e+26,0.732,1.445586,0.0,1.794407e+17,1.208942e-10
3,2.028704e+25,3.774162e+07,1.450997e+07,90.088137,3.506012e+26,0.732,1.445589,0.0,1.794400e+17,1.208929e-10
4,4.808770e+25,5.032217e+07,1.450993e+07,90.087811,8.310454e+26,0.732,1.445594,0.0,1.794389e+17,1.208910e-10
...,...,...,...,...,...,...,...,...,...,...
9912,1.991121e+33,4.739400e+10,6.892412e+03,0.000088,3.565993e+33,0.732,0.401054,1.0,8.357322e+07,1.617101e-47
9913,1.991121e+33,4.739403e+10,6.887369e+03,0.000088,3.565993e+33,0.732,0.401053,1.0,8.342037e+07,1.602843e-47
9914,1.991121e+33,4.739406e+10,6.882327e+03,0.000088,3.565993e+33,0.732,0.401052,1.0,8.326769e+07,1.588700e-47
9915,1.991121e+33,4.739409e+10,6.877285e+03,0.000088,3.565993e+33,0.732,0.401051,1.0,8.311518e+07,1.574673e-47


In [10]:
epoch = 0
_ = df.to_csv(line_terminator='\n')
filename = "FromCenter/4epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

FromCenter/4epoch00.csv


### Поиск комбинации центральных плотности и температуры, дающих решение с массой равной солнечной. (Второй вариант)

Для интегрирования уравнений из центра надо задать плотность и температуру в центре, но только одна их комбинации соответствует решению с массой, равной солнечной (при фиксированном хим. составе)

Альтернативный поиск (в обратном порядке)

Внешний цикл по плотности, внутренний цикл - по температуре. При фиксированной плотности постепенно понижаем температуру, пока не появится стабильное решение с массой < 1. После этого возвращаемся на 1 шаг назад по температуре, уменьшаем шаг по температуре в 10 раз и повторям процедуру. Так делаем пока шаг по температуре не станет меньше 0.01 градуса.

In [ ]:
%%time
# Another order of fit - density outside
dim = 11
for j in range(dim) :
    den0 = 94 + 0.1*j # 94.4 + 0.1*j
    step = 1e6 # step for Temperature
    exc = True
    T0 = 15
    if j > 0:
        df_prev = df.copy()
    while step > 1e-2 :    
        for i in range(10) :
            try :
                n, M, R, L, T, df = model_R(den0, T0, debug=False)
            except Exception as error:
#                 print(T0, den0, 'Caught this error: ' + repr(error))
                exc = True
                T0 -= step
                break
            if exc :
                exc = False
                print('\tT0=%.5f D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
                T0 += step
                step /= 10.
                break
            
    print('# T0=%.5f D0=%.2f n=%d M=%.4f R=%.4f L=%.3f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
    if M/M_sol > 1. :
#         df = df_prev
        break
print("Done!")

In [ ]:
# df = df_prev
df

In [ ]:
_ = df.to_csv(line_terminator='\n')
filename = "FromCenter/2epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

In [30]:
T0=1.33000e+07 
den0=94.4000000000
n, M, R, L, T, df = model_R(den0, T0, debug=False)
n, M/M_sol, R/R_sol, L/L_sol, T

(11049,
 0.8772552888605576,
 0.5009900889800694,
 0.6252592215835585,
 7252.863270233664)